In [ ]:
#hide
from nbdev.showdoc import *
from splashy.finite_diff import get_stencil, apply_stencil
import matplotlib
matplotlib.use("nbagg")

In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Problem definition

In [ ]:
# define problem

f_0 = 10  # center freq in hertz for source time func

x_max = 1_000  # length
x_min = -1_000
dx = .1  # spatial resolution

t_max = 10  # max time 
t_0 = 4 / f_0
dt = 0.00025  # time steps

velocity = 343  # velocity, m/s

space = np.arange(x_min, x_max, step=dx)
time = np.arange(0, t_max, step=dt)

output = np.zeros((len(time), len(space)))

In [ ]:
assert (dx / dt ) > velocity, 'solution doesnt meet CFL criteron'

## Source Time function

First we define the source time function to use in the simulation.

Apparently the derivative of a gausian is common, so we will use that.

$$
f(t) = -8f_0(t-t_0)*exp(-16f_0^2 (t - t_0)^2)
$$


In [ ]:
# source time function
def get_source_time(time, f0, t0=0):
    """Given a time vector return the source time function."""
    c1 = -8 * f0 * (time - t0)
    c2 = -16 * f0 **2
    c3 = (time - t0) ** 2
    return c1 * np.exp(c2 * c3)


In [ ]:
source_time = get_source_time(time, f_0, t_0)
source_spec = np.fft.rfft(source_time)
source_freq = np.fft.rfftfreq(len(time), d=dt)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.plot(time, source_time)
ax1.set_xlabel('time (seconds)')
ax1.set_ylabel('source output')
ax1.set_xlim(t_0 - .1, t_0 + .1)

ax2.plot(source_freq, np.abs(source_spec))
ax2.set_xlabel('frequency')
ax2.set_xlim(-10, 100)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Numerical Solution (Finite Differences Method)

The acoustic wave equation in 1D with constant density 

$$
\partial^2_t p(x,t) \ = \ c(x)^2 \partial_x^2 p(x,t) + s(x,t)
$$

with pressure $p$, acoustic velocity $c$, and source term $s$ contains two second derivatives that can be approximated with a difference formula such as

$$
\partial^2_t p(x,t) \ \approx \ \frac{p(x,t+dt) - 2 p(x,t) + p(x,t-dt)}{dt^2} 
$$

and equivalently for the space derivative. Injecting these approximations into the wave equation allows us to formulate the pressure p(x) for the time step $t+dt$ (the future) as a function of the pressure at time $t$ (now) and $t-dt$ (the past). This is called an explicit scheme allowing the $extrapolation$ of the space-dependent field into the future only looking at the nearest neighbourhood.

We replace the time-dependent (upper index time, lower indices space) part by

$$
 \frac{p_{i}^{n+1} - 2 p_{i}^n + p_{i}^{n-1}}{\mathrm{d}t^2} \ = \ c^2 ( \partial_x^2 p) \ + s_{i}^n
$$

solving for $p_{i}^{n+1}$.

The extrapolation scheme is

$$
p_{i}^{n+1} \ = \ c_i^2 \mathrm{d}t^2 \left[ \partial_x^2 p \right]
+ 2p_{i}^n - p_{i}^{n-1} + \mathrm{d}t^2 s_{i}^n
$$

The  space derivatives are determined by 

$$
\partial_x^2 p \ = \ \frac{p_{i+1}^{n} - 2 p_{i}^n + p_{i-1}^{n}}{\mathrm{d}x^2}
$$

In [ ]:
d2x_stencil = get_stencil(2, 3, dx=dx)
space_delta = np.zeros_like(space)
delta_ind = np.argmin(np.abs(space))
space_delta[delta_ind] = 1 / dx


In [ ]:
# iterate each timestep in output
for n in range(1, len(output) -1):
    # get current, past, and future pressure state (sliced by time)
    current_output = output[n]
    previous_output = output[n - 1]
    # apply stencil to get second spactial derivative
    
    d2x = apply_stencil(current_output, d2x_stencil)
    if not np.isfinite(d2x).all():
        break
    
    
    pde = velocity ** 2 * dt ** 2 * d2x + 2 * current_output - previous_output
    source = space_delta * source_time[n]
    future_ouput = pde  + source 
    future_ouput[:2] = 0
    future_ouput[-2:] = 0
    if not np.isfinite(future_ouput).all():
        break
    output[n + 1] = future_ouput

# Plot results
Now we plot the results of the simulation in a simple animation.

In [ ]:
# setup figure
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1,1,1)
ax.set_xlim(x_min, x_max)
ax.set_ylim(-1, 1)
ax.set_xlabel('X (m)')
ax.set_ylabel('Pressure Amplitude')
ax.set_title('Time (0)')
ax.axvline(0, color='red')
up31, = ax.plot(space, output[0])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
time_inds = np.arange(0, len(time), step=150)
# start_ind = np.where(time==t_0)[0]
# time_inds = np.arange(start_ind, start_ind + 500, )

In [ ]:
for n in time_inds:
    current_output = output[n]
    current_time = time[n]
    up31.set_ydata(current_output)
    ax.set_title(f'Time ({current_time:.2f})')
    ax.set_ylim(current_output.min(), current_output.max())
    plt.gcf().canvas.draw()


/home/derrick/anaconda3/envs/waves/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Attempting to set identical bottom == top == 0.0 results in singular transformations; automatically expanding.
  
